In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
import threading
import re

In [2]:
jumlah_index = 200
threads_link = []
links = []
results = []
threads = []
keywords='ganjar'

In [3]:
def scrape_links(page_number,keywords):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
    }
    url = f"https://www.cnbcindonesia.com/search?query={keywords}&p={page_number}&kanal=&tipe=&date="
    response = requests.get(url,headers=headers)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        articles = soup.find_all('article')
        
        page_links = []
        for article in articles:
            link = article.find('a')['href']
            page_links.append(link)
        
        print(f"Scraped {len(page_links)} links from page {page_number}")
        return page_links
    else:
        print(f"Failed to retrieve data from {url}")        

In [4]:
for page_number in range(1, jumlah_index + 1):
    thread = threading.Thread(target=lambda p=page_number: links.extend(scrape_links(p,keywords)))
    thread.start()
    threads_link.append(thread)

for thread in threads_link:
    thread.join()
print("Total Links:", len(links))

Scraped 0 links from page 138
Scraped 12 links from page 84
Scraped 12 links from page 41
Scraped 4 links from page 131
Scraped 0 links from page 136
Scraped 12 links from page 29
Scraped 12 links from page 25
Scraped 12 links from page 75
Scraped 12 links from page 81
Scraped 12 links from page 101
Scraped 12 links from page 77
Scraped 12 links from page 17
Scraped 0 links from page 140
Scraped 12 links from page 10
Scraped 0 links from page 139
Scraped 0 links from page 137
Scraped 0 links from page 133
Scraped 12 links from page 83
Scraped 0 links from page 144
Scraped 12 links from page 54
Scraped 12 links from page 50
Scraped 12 links from page 100
Scraped 12 links from page 19
Scraped 12 links from page 36
Scraped 0 links from page 143
Scraped 12 links from page 2
Scraped 12 links from page 13
Scraped 12 links from page 3
Scraped 12 links from page 38
Scraped 12 links from page 129
Scraped 12 links from page 105
Scraped 12 links from page 15
Scraped 12 links from page 30
Scraped 

In [5]:
results = []
def scrape_url(url,keywords):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            
            
            # Judul Berita
            title_elem = soup.find('h1')
            if title_elem:
                title_text = title_elem.text.strip()
            else:
                title_text = "Title not found"
            # Author berita
            author_elem = soup.find('div', {"class": "author"})
            if author_elem:
                author_text = author_elem.get_text()
                author_text = author_text.split('-')[1].strip()
                author_text = author_text.split(',')[0].strip()
            else:
                author_text = "Author not found"     
            # tanggal berita
            date_elem = soup.find('div', {"class": "date"})
            if date_elem:
                date_text = date_elem.text.strip()
            else:
                date_text = "Date not found"
            #     # Category berita
            category_elements = soup.find('ul', {"class": "breadcrumb"})
            if category_elements:
                category_text = category_elements.find_all('li')
                category_text= category_text[2].get_text()
            else:
                category_text = "Category not found"
            #     # Content Berita
            body_elem = soup.find('div', {"class": "detail_text"})
            
            if body_elem:
                content_elem = body_elem.find_all('p')
                content_text = ""
                for p in content_elem:
                    content_text += p.text.strip() + "\n"
                
                if content_text.strip():
                    content_text=content_text
                else:
                    content_text ="Content not found"
            else:
              content_text ="Content not found"
            
            nama_berita_match = re.search(r'https://www\.(\w+)\.com/', url)
            if nama_berita_match:
                nama_berita = nama_berita_match.group(1)
            else:
                nama_berita = "Nama_berita not found"

            results.append({'title': title_text,
                            'keywords': keywords,
                            'author' : author_text,
                            'category':category_text,
                            'date': date_text,
                            'content' : content_text,
                            'nama_berita' : nama_berita,
                            'link' : url})
        else:
            print(f"Failed to retrieve data from {url}")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL '{url}': {e}")
    except Exception as e:
        print(f"Error processing URL '{url}': {e}")

In [6]:
for url in links:
    thread = threading.Thread(target=scrape_url, args=(url,keywords))
    thread.start()
    threads.append(thread)
    
for thread in threads:
    thread.join()

In [7]:
df = pd.DataFrame(results)
print('hasil scrapping',len(results))
df.head(10)

hasil scrapping 1504


,title,keywords,author,category,date,content,nama_berita,link
0,Video: Prabowo Pendamping Ganjar? Ini Jawaban ...,ganjar,CNBC Indonesia TV,Video News,22 April 2023 17:30,"Jakarta, CNBC Indonesia - Kunjungan Prabowo ke...",cnbcindonesia,https://www.cnbcindonesia.com/news/20230422165...
1,"Resmi Jadi Capres, Jangan Kaget Segini Harta G...",ganjar,Susi Setiawati,Berita Research,25 April 2023 16:35,"Jakarta, CNBC Indonesia - Ketua Umum Partai De...",cnbcindonesia,https://www.cnbcindonesia.com/research/2023042...
2,"Sanggupkah Ganjar Pranowo Akhiri ""Kutukan"" IHS...",ganjar,mae,Berita Research,25 April 2023 15:00,"Jakarta, CNBC Indonesia - Bursa Indonesia akan...",cnbcindonesia,https://www.cnbcindonesia.com/research/2023042...
3,"Ganjar Pranowo Capres, Seberapa Besar Dampakny...",ganjar,Putu Agus Pransuamitra,Berita Market,26 April 2023 08:35,"Jakarta, CNBC Indonesia - Pasar keuangan Indon...",cnbcindonesia,https://www.cnbcindonesia.com/market/202304260...
4,"Mirip Jokowi, Ganjar Pranowo Effect Bisa Bikin...",ganjar,mae,Berita Research,26 April 2023 06:02,"Jakarta, CNBC Indonesia - Pasar keuangan Indon...",cnbcindonesia,https://www.cnbcindonesia.com/research/2023042...
5,"Rupiah Melemah Sentuh Rp 14.900/US$, Tak Ada G...",ganjar,Putu Agus Pransuamitra,Berita Market,26 April 2023 09:09,"Jakarta, CNBC Indonesia - Rupiah melemah melaw...",cnbcindonesia,https://www.cnbcindonesia.com/market/202304260...
6,Jos! IHSG Dibuka Menguat Setelah Libur Panjang,ganjar,Muhammad Azwar,Berita Market,26 April 2023 09:12,"Jakarta, CNBC Indonesia- Indeks Harga Saham Ga...",cnbcindonesia,https://www.cnbcindonesia.com/market/202304260...
7,Jreng! Jokowi Ungkap Sosok Ini Cocok Jadi Cawa...,ganjar,Arrijal Rachman,Berita,22 April 2023 22:00,"Jakarta, CNBC Indonesia - Ketua Umum PDI Perju...",cnbcindonesia,https://www.cnbcindonesia.com/news/20230422205...
8,Kejutan! Cawapres Anies Diajak Gabung Jadi Tim...,ganjar,Wiji Nur Hayat,Berita,02 August 2023 06:15,"Jakarta, CNBC Indonesia - Salah satu kandidat ...",cnbcindonesia,https://www.cnbcindonesia.com/news/20230801202...
9,"Anies Vs Ganjar dalam Penanganan Banjir, Siapa...",ganjar,Tim Riset,Berita Research,23 April 2023 14:30,"Jakarta, CNBC Indonesia - Pemilihan Umum (Pemi...",cnbcindonesia,https://www.cnbcindonesia.com/research/2023042...


In [8]:
filtered_df = df[df['content'].str.contains(keywords, case=False)|
                 df['title'].str.contains(keywords, case=False)]
print('hasil filter',len(filtered_df))
filtered_df.head(10)

hasil filter 1392


,title,keywords,author,category,date,content,nama_berita,link
0,Video: Prabowo Pendamping Ganjar? Ini Jawaban ...,ganjar,CNBC Indonesia TV,Video News,22 April 2023 17:30,"Jakarta, CNBC Indonesia - Kunjungan Prabowo ke...",cnbcindonesia,https://www.cnbcindonesia.com/news/20230422165...
1,"Resmi Jadi Capres, Jangan Kaget Segini Harta G...",ganjar,Susi Setiawati,Berita Research,25 April 2023 16:35,"Jakarta, CNBC Indonesia - Ketua Umum Partai De...",cnbcindonesia,https://www.cnbcindonesia.com/research/2023042...
2,"Sanggupkah Ganjar Pranowo Akhiri ""Kutukan"" IHS...",ganjar,mae,Berita Research,25 April 2023 15:00,"Jakarta, CNBC Indonesia - Bursa Indonesia akan...",cnbcindonesia,https://www.cnbcindonesia.com/research/2023042...
3,"Ganjar Pranowo Capres, Seberapa Besar Dampakny...",ganjar,Putu Agus Pransuamitra,Berita Market,26 April 2023 08:35,"Jakarta, CNBC Indonesia - Pasar keuangan Indon...",cnbcindonesia,https://www.cnbcindonesia.com/market/202304260...
4,"Mirip Jokowi, Ganjar Pranowo Effect Bisa Bikin...",ganjar,mae,Berita Research,26 April 2023 06:02,"Jakarta, CNBC Indonesia - Pasar keuangan Indon...",cnbcindonesia,https://www.cnbcindonesia.com/research/2023042...
5,"Rupiah Melemah Sentuh Rp 14.900/US$, Tak Ada G...",ganjar,Putu Agus Pransuamitra,Berita Market,26 April 2023 09:09,"Jakarta, CNBC Indonesia - Rupiah melemah melaw...",cnbcindonesia,https://www.cnbcindonesia.com/market/202304260...
6,Jos! IHSG Dibuka Menguat Setelah Libur Panjang,ganjar,Muhammad Azwar,Berita Market,26 April 2023 09:12,"Jakarta, CNBC Indonesia- Indeks Harga Saham Ga...",cnbcindonesia,https://www.cnbcindonesia.com/market/202304260...
7,Jreng! Jokowi Ungkap Sosok Ini Cocok Jadi Cawa...,ganjar,Arrijal Rachman,Berita,22 April 2023 22:00,"Jakarta, CNBC Indonesia - Ketua Umum PDI Perju...",cnbcindonesia,https://www.cnbcindonesia.com/news/20230422205...
8,Kejutan! Cawapres Anies Diajak Gabung Jadi Tim...,ganjar,Wiji Nur Hayat,Berita,02 August 2023 06:15,"Jakarta, CNBC Indonesia - Salah satu kandidat ...",cnbcindonesia,https://www.cnbcindonesia.com/news/20230801202...
9,"Anies Vs Ganjar dalam Penanganan Banjir, Siapa...",ganjar,Tim Riset,Berita Research,23 April 2023 14:30,"Jakarta, CNBC Indonesia - Pemilihan Umum (Pemi...",cnbcindonesia,https://www.cnbcindonesia.com/research/2023042...


Save File

In [9]:
# current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
# excel_file_name = f'../tempat_simpan_nasional/cnbc_{keywords}_{current_datetime}.xlsx'
# filtered_df.to_excel(excel_file_name, index=False)

# print(f'Data has been saved to {excel_file_name}')